This notebook takes the generated text, cleans it as necessary, creates the embeddings, and performs the dimension reduction.

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git


In [ ]:
#!pip install -U yellowbrick

In [ ]:
!pip install umap-learn

In [ ]:
#import lucem_illud
import umap

In [ ]:
import gensim
import pandas
import sklearn

In [ ]:
import numpy as np

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en")

### Import Data and Clean

In [ ]:
data_d = pd.read_csv("/content/drive/My Drive/Final Project/Data/text-generation-results/quarterly-democrats.csv")
data_s = pd.read_csv("/content/drive/My Drive/Final Project/Data/text-generation-results/quarterly-socialism.csv")
data_c17 = pd.read_csv("/content/drive/My Drive/Final Project/Data/text-generation-results/2017-conservative.csv")
data_c18 = pd.read_csv("/content/drive/My Drive/Final Project/Data/text-generation-results/2018-conservative.csv")
data_c19 = pd.read_csv("/content/drive/My Drive/Final Project/Data/text-generation-results/2019-conservative.csv")

In [ ]:
#Socialism
for i in range(len(data_s)):
    seq_len = len(data_s.Prompt[i].split(" "))
    data_s.Result[i] = data_s.Result[i].replace("\n\n", " \n\n ")
    data_s.Result[i] = " ".join(data_s.Result[i].split(" ")[seq_len:])

#Democrats
for i in range(len(data_d)):
    seq_len = len(data_d.Prompt[i].split(" "))
    data_d.Result[i] = data_d.Result[i].replace("\n\n", " \n\n ")
    data_d.Result[i] = " ".join(data_d.Result[i].split(" ")[seq_len:])

#Conservative 2017
for i in range(len(data_c17)):
    seq_len = len(data_c17.Prompt[i].split(" "))
    data_c17.Result[i] = data_c17.Result[i].replace("\n\n", " \n\n ")
    data_c17.Result[i] = " ".join(data_c17.Result[i].split(" ")[seq_len:])

#Conservative 2018
for i in range(len(data_c18)):
    seq_len = len(data_c18.Prompt[i].split(" "))
    data_c18.Result[i] = data_c18.Result[i].replace("\n\n", " \n\n ")
    data_c18.Result[i] = " ".join(data_c18.Result[i].split(" ")[seq_len:])

#Conservative 2019
for i in range(len(data_c19)):
    seq_len = len(data_c19.Prompt[i].split(" "))
    data_c19.Result[i] = data_c19.Result[i].replace("\n\n", " \n\n ")
    data_c19.Result[i] = " ".join(data_c19.Result[i].split(" ")[seq_len:])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib

In [ ]:
# Add label column
data_s["label"] = "socialism"
data_d["label"] = "democrats"
data_c17["label"] = "conservative"
data_c18["label"] = "conservative"
data_c19["label"] = "conservative"

In [ ]:
# Only keep q1 and q2 for 2019 conservative
data_c19 = data_c19.iloc[0:540, :]

In [ ]:
#for loop below
dfs = ["data_c17", "data_c18", "data_c19", "data_d", "data_s"]

In [ ]:
#create quarter column for conservative data
data_c17['Quarter'] = 0
for i in range(len(data_c17)):
    if data_c17.Month[i] >= 1 and data_c17.Month[i] <= 3:
        data_c17.Quarter[i] = 1
    elif data_c17.Month[i] > 3 and data_c17.Month[i] <= 6:
        data_c17.Quarter[i] = 2
    elif data_c17.Month[i] > 6 and data_c17.Month[i] <= 9:
        data_c17.Quarter[i] = 3
    else:
        data_c17.Quarter[i] = 4
del data_c17['Month']

data_c18['Quarter'] = 0
for i in range(len(data_c18)):
    if data_c18.Month[i] >= 1 and data_c18.Month[i] <= 3:
        data_c18.Quarter[i] = 1
    elif data_c18.Month[i] > 3 and data_c18.Month[i] <= 6:
        data_c18.Quarter[i] = 2
    elif data_c18.Month[i] > 6 and data_c18.Month[i] <= 9:
        data_c18.Quarter[i] = 3
    else:
        data_c18.Quarter[i] = 4
del data_c18['Month']

data_c19['Quarter'] = 0
for i in range(len(data_c19)):
    if data_c19.Month[i] >= 1 and data_c19.Month[i] <= 3:
        data_c19.Quarter[i] = 1
    elif data_c19.Month[i] > 3 and data_c19.Month[i] <= 6:
        data_c19.Quarter[i] = 2
    elif data_c19.Month[i] > 6 and data_c19.Month[i] <= 9:
        data_c19.Quarter[i] = 3
    else:
        data_c19.Quarter[i] = 4
del data_c19['Month']

In [ ]:
data = data_d.append(data_s.append(data_c17.append(data_c18.append(data_c19))))

In [ ]:
data

,Unnamed: 0,Quarter,Year,Prompt,Result,label
0,0,1,2017,When it comes to data privacy,it's important to remember that the government...,democrats
1,1,1,2017,When it comes to data privacy,it's important to understand that the governme...,democrats
2,2,1,2017,When it comes to data privacy,there are a lot of things that can be done to ...,democrats
3,3,1,2017,When it comes to data privacy,there are a lot of people who are not happy wi...,democrats
4,4,1,2017,When it comes to data privacy,it's important to keep in mind that the govern...,democrats
...,...,...,...,...,...,...
535,535,2,2019,"When it comes to Facebook, the government should",be able to ban it. \n\n The government shouldn...,conservative
536,536,2,2019,"When it comes to Facebook, the government should",not be able to control the content of its user...,conservative
537,537,2,2019,"When it comes to Facebook, the government should",be able to regulate it. \n\n The government sh...,conservative
538,538,2,2019,"When it comes to Facebook, the government should",have the power to ban it. \n\n The government ...,conservative


## D2V 50D with UMAP


In [ ]:
import spacy
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
import numpy as np

In [ ]:
prompts = data['Prompt'].unique()

In [ ]:
nlp = spacy.load('en')

#### Prompt 0

In [ ]:
data_0 = data[data.Prompt == prompts[0]]

In [ ]:
texts, article = [], []
for text in data_0['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=50)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

UMAP for dimension reduction

In [ ]:
reducer = umap.UMAP()

In [ ]:
embedding_umap = reducer.fit_transform(d2v_results)

In [ ]:
# Add D2V results back into DF
data_0_embeddings = data_0

In [ ]:
data_0_embeddings['umap_x'] = embedding_umap[:, 0]
data_0_embeddings['umap_y'] = embedding_umap[:, 1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Prompt 1

In [ ]:
data_1 = data[data.Prompt == prompts[1]]

In [ ]:
texts, article = [], []
for text in data_1['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=50)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

UMAP for dimension reduction

In [ ]:
reducer = umap.UMAP()

In [ ]:
embedding_umap = reducer.fit_transform(d2v_results)

In [ ]:
# Add D2V results back into DF
data_1_embeddings = data_1

In [ ]:
data_1_embeddings['umap_x'] = embedding_umap[:, 0]
data_1_embeddings['umap_y'] = embedding_umap[:, 1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Prompt 2

In [ ]:
data_2 = data[data.Prompt == prompts[2]]

In [ ]:
texts, article = [], []
for text in data_2['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=50)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

UMAP for dimension reduction

In [ ]:
reducer = umap.UMAP()

In [ ]:
embedding_umap = reducer.fit_transform(d2v_results)

In [ ]:
# Add D2V results back into DF
data_2_embeddings = data_2

In [ ]:
data_2_embeddings['umap_x'] = embedding_umap[:, 0]
data_2_embeddings['umap_y'] = embedding_umap[:, 1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Prompt 3

In [ ]:
data_3 = data[data.Prompt == prompts[3]]

In [ ]:
texts, article = [], []
for text in data_3['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=50)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

UMAP for dimension reduction

In [ ]:
reducer = umap.UMAP()

In [ ]:
embedding_umap = reducer.fit_transform(d2v_results)

In [ ]:
# Add D2V results back into DF
data_3_embeddings = data_3

In [ ]:
data_3_embeddings['umap_x'] = embedding_umap[:, 0]
data_3_embeddings['umap_y'] = embedding_umap[:, 1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Prompt 4

In [ ]:
data_4 = data[data.Prompt == prompts[4]]

In [ ]:
texts, article = [], []
for text in data_4['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=50)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

UMAP for dimension reduction

In [ ]:
reducer = umap.UMAP()

In [ ]:
embedding_umap = reducer.fit_transform(d2v_results)

In [ ]:
# Add D2V results back into DF
data_4_embeddings = data_4

In [ ]:
data_4_embeddings['umap_x'] = embedding_umap[:, 0]
data_4_embeddings['umap_y'] = embedding_umap[:, 1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Prompt 5

In [ ]:
data_5 = data[data.Prompt == prompts[5]]

In [ ]:
texts, article = [], []
for text in data_5['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=50)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

UMAP for dimension reduction

In [ ]:
reducer = umap.UMAP()

In [ ]:
embedding_umap = reducer.fit_transform(d2v_results)

In [ ]:
# Add D2V results back into DF
data_5_embeddings = data_5

In [ ]:
data_5_embeddings['umap_x'] = embedding_umap[:, 0]
data_5_embeddings['umap_y'] = embedding_umap[:, 1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Prompt 6

In [ ]:
data_6 = data[data.Prompt == prompts[6]]

In [ ]:
texts, article = [], []
for text in data_6['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=50)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

UMAP for dimension reduction

In [ ]:
reducer = umap.UMAP()

In [ ]:
embedding_umap = reducer.fit_transform(d2v_results)

In [ ]:
# Add D2V results back into DF
data_6_embeddings = data_6

In [ ]:
data_6_embeddings['umap_x'] = embedding_umap[:, 0]
data_6_embeddings['umap_y'] = embedding_umap[:, 1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Prompt 7

In [ ]:
data_7 = data[data.Prompt == prompts[7]]

In [ ]:
texts, article = [], []
for text in data_7['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=50)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

UMAP for dimension reduction

In [ ]:
reducer = umap.UMAP()

In [ ]:
embedding_umap = reducer.fit_transform(d2v_results)

In [ ]:
# Add D2V results back into DF
data_7_embeddings = data_7

In [ ]:
data_7_embeddings['umap_x'] = embedding_umap[:, 0]
data_7_embeddings['umap_y'] = embedding_umap[:, 1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Prompt 8

In [ ]:
data_8 = data[data.Prompt == prompts[8]]

In [ ]:
texts, article = [], []
for text in data_8['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=50)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

UMAP for dimension reduction

In [ ]:
reducer = umap.UMAP()

In [ ]:
embedding_umap = reducer.fit_transform(d2v_results)

In [ ]:
# Add D2V results back into DF
data_8_embeddings = data_8

In [ ]:
data_8_embeddings['umap_x'] = embedding_umap[:, 0]
data_8_embeddings['umap_y'] = embedding_umap[:, 1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Combine Embedding Data

In [ ]:
final_df = data_0_embeddings.append(data_1_embeddings.append(data_2_embeddings.append(data_3_embeddings.append(data_4_embeddings.append(data_5_embeddings.append(data_6_embeddings.append(data_7_embeddings.append(data_8_embeddings))))))))

In [ ]:
final_df.to_csv("/content/drive/My Drive/Final Project/Data/Embeddings/embeddings-50d-umap-all-subreddits.csv")